In [ ]:
import pandas as pd
from binance import Client
import snscrape.modules.twitter as sntwitter
import seaborn as sns
from deep_translator import GoogleTranslator
import datetime as dt
from gensim import corpora
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer
from datetime import date




import numpy as np, pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
sns.set() # use seaborn plotting style

In [ ]:
PUBLIC_KEY = 'WhihdXQmvY9QAObIELyVGAI3h1nvJZDYg1B68H7n0MHn0bISKXMwHozMcxMbaJ0Z'
PRIVATE_KEY = '2auNidRhbGrAG06UWK0MPcPYJtBj9fMFKpAWJsyjTACJLHHIdOHVbsLj8MZopJJ4'
client = Client(PUBLIC_KEY, PRIVATE_KEY)
tickers = client.get_all_tickers()
df = pd.DataFrame()
for coins in client.get_all_tickers():
    if((coins["symbol"][-4:]=="BUSD")):
        historical_data = client.get_historical_klines(
            coins["symbol"], Client.KLINE_INTERVAL_1DAY, '19 July 2022')
        coin_df = pd.DataFrame(historical_data,columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time',
                'Quote Asset Volume', 'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore'])
        coin_df["COIN"]=coins["symbol"][:-4]
        df = pd.concat([df,coin_df],axis=0)
df['Open Time'] = pd.to_datetime(df['Open Time']/1000, unit='s')
df['Close Time'] = pd.to_datetime(df['Close Time']/1000, unit='s')
numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume',
                   'Quote Asset Volume', 'TB Base Volume', 'TB Quote Volume']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, axis=1)
df["Change"] = ((df["High"]-df["Low"])/df["Low"]).apply(lambda x: x*100)


In [ ]:
df = df.sort_values(by="Change",ascending=False)
df

In [ ]:
tweets_list1 = []
username_not_needed = ['dashfit_signal','moon_or_earth', 'Crypto3OT', 'abnormal_crypto']
begin_date = dt.date(2022, 6, 18)
end_date = dt.date(2022, 7, 18)
# >30 is not good do something else
for coin in df.loc[df["Change"] > 30, "COIN"]:
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper('#'+coin, begin_date, end_date).get_items()):
        if ((i < 1000)):
            if((tweet.user.username not in username_not_needed)):
                (tweets_list1.append([tweet.date, tweet.user.username,coin,GoogleTranslator('auto', 'en').translate(tweet.content)]))
        else:
            break
tweets_df1 = pd.DataFrame(tweets_list1, columns=['Datetime', 'Username', 'Coin', 'Text'])


In [ ]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
# current_date =date.today()
def text_cleaning(text):
    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()

tweets_df1["Cleaned Text"]= tweets_df1["Text"].apply(text_cleaning)

In [ ]:
# tweets_df1 = pd.read_csv("19.csv")
# tweets_df1

In [ ]:
df = pd.read_csv("Untitled spreadsheet - 11.csv")
df1 = df.drop(columns=["Unnamed: 0","Username","Coin","Text","Datetime"])
X = df1.iloc[:,1]
Y = df1.iloc[:,0]
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=.2)
model = make_pipeline(TfidfVectorizer(lowercase=False), MultinomialNB())
model.fit(X_train, Y_train)
X


In [ ]:
def my_predictions(my_sentence, model):
    prediction = model.predict([my_sentence])
    return prediction
result=list()
# tweets_df1["Cleaned Text"].apply(lambda x: x.lower())
for i in tweets_df1["Cleaned Text"]:
    result.append(my_predictions(str(i),model))
tweets_df1["Result"]=result
tweets_df1

In [ ]:
tweets_df1

In [ ]:
tweets_df1.to_csv("19 test.csv")

In [ ]:
sns.set(rc={'figure.figsize': (70, 20)})
sns.set_theme(style="darkgrid")
ax = sns.countplot(x="Username", data=tweets_df1)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)


In [ ]:
tweets_df1["Username"].value_counts()

In [ ]:
tweets_df1["Cleaned Text"]

In [ ]:

#clean the data
# stop = set(stopwords.words('english'))
# exclude = set(string.punctuation)
# lemma = WordNetLemmatizer()

# def clean(text):
#     stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
#     punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
#     normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
#     return normalized.split()

# df['text_clean']=df['text'].apply(clean)
#create dictionary
# dictionary = corpora.Dictionary(tweets_df1['Cleaned Text'])
#Total number of non-zeroes in the BOW matrix (sum of the number of unique words per document over the entire corpus).
# print(dictionary.most_common())

In [ ]:
df = df.sort_values(by=df["Date"])
df = df[df["Change"]>30]
for i in list(set(df["COIN"])):
    print(i)
    # dict(zip(i, values))